# Mounting the Google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cloning the GitHub repository

In [ ]:
!git clone https://github.com/NishantPatel18/RoadCrackDetection.git

Cloning into 'RoadCrackDetection'...
remote: Enumerating objects: 121487, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 121487 (delta 20), reused 4 (delta 1), pack-reused 121451
Receiving objects: 100% (121487/121487), 2.65 GiB | 30.95 MiB/s, done.
Resolving deltas: 100% (10213/10213), done.
Checking out files: 100% (179989/179989), done.


# Training

In [2]:
%cd /content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD

import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T

/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD


In [3]:
pd.read_csv("/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD/all_cities.csv")

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Numazu_20170906095801.jpg,600,600,Crack,69,371,600,576
1,Numazu_20170906144736.jpg,600,600,Crack,79,273,214,540
2,Adachi_20170906093840.jpg,600,600,Crack,1,346,203,598
3,Adachi_20170921160022.jpg,600,600,Crack,324,343,600,565
4,Adachi_20170921160022.jpg,600,600,Crack,1,293,116,499
...,...,...,...,...,...,...,...,...
15452,Chiba_20170913131136.jpg,600,600,Crack,89,498,448,565
15453,Nagakute_20170912134838.jpg,600,600,Crack,93,353,248,599
15454,Numazu_20170906144333.jpg,600,600,Crack,201,309,551,586
15455,Adachi_20170908141650.jpg,600,600,Crack,6,203,594,301


In [4]:
def parse_one_annot(path_to_data_file, filename):
    data = pd.read_csv(path_to_data_file)
    boxes_array = data[data["filename"] == filename][["xmin", "ymin", "xmax", "ymax"]].values
    class_name = data[data["filename"] == filename][["class"]].values

    return boxes_array, class_name

def get_class_number(class_input):
    class_number = 0
    if (class_input == ['Crack']):
        class_number = 1
    else:
        class_number = 0

    return class_number

class RoadDamageDataset(torch.utils.data.Dataset):
    def __init__(self, root, data_file, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = sorted(os.listdir(os.path.join(root)))
        self.path_to_data_file = data_file

    def __getitem__(self, idx):
        # load images and bounding boxes
        img_path = os.path.join(self.root, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        box_list, class_name = parse_one_annot(self.path_to_data_file,
                                               self.imgs[idx])
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        class_list = []
        #print(class_name)

        for name in class_name:
            #print(name)
            number = get_class_number(name)
            #print(number)
            class_list.append(number)

        #print(class_list)
        class_list_numpy = np.array(class_list)
        labels = torch.as_tensor(class_list_numpy, dtype=torch.int64)
        # labels = torch.ones((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        # print(target)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    def __len__(self):
        return len(self.imgs)

In [5]:
dataset = RoadDamageDataset(root="/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/RoadDamageDataset/All_cities/Images",
                         data_file="/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD/all_cities.csv")
dataset.__getitem__(0)

(<PIL.Image.Image image mode=RGB size=600x600 at 0x7F029F168D10>,
 {'area': tensor([20989.]),
  'boxes': tensor([[ 87., 281., 226., 432.]]),
  'image_id': tensor([0]),
  'iscrowd': tensor([0]),
  'labels': tensor([1])})

In [6]:
def get_model(num_classes):
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [7]:
# use our dataset and defined transformations
dataset = RoadDamageDataset(root="/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/RoadDamageDataset/All_cities/Images",
                         data_file="/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD/all_cities.csv",
                         transforms=get_transform(train=True))

dataset_test = RoadDamageDataset(
    root="/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/RoadDamageDataset/All_cities/Images",
    data_file="/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD/all_cities.csv",
    transforms=get_transform(train=False))

In [8]:
# split the dataset in train and test set
torch.manual_seed(1)
num_total_for_train_test = 9892
# 30%
num_test = 2968
# num_test = 10
num_train = num_total_for_train_test - num_test
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:num_train])
dataset_test = torch.utils.data.Subset(dataset_test, indices[num_train:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=14, shuffle=True, num_workers=4,
                                          collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=14, shuffle=False, num_workers=4,
                                               collate_fn=utils.collate_fn)

print("We have: {} examples, {} are training and {} are testing".format(len(indices), len(dataset), len(dataset_test)))

device = torch.device('cuda')

We have: 9892 examples, 6924 are training and 2968 are testing


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
# our dataset has 2 classes which are Crack and NO(negative class)
num_classes = 2

# get the model using our helper function
model = get_model(num_classes)

# move model to the right device
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [10]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [11]:
# let's train it for 3 epochs
num_epochs = 3

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    # change print freq to display 1 by 1
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

    # update the learning rate
    lr_scheduler.step()

    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [1]  [  0/495]  eta: 1:27:20  lr: 0.000015  loss: 0.9003 (0.9003)  loss_classifier: 0.5554 (0.5554)  loss_box_reg: 0.0778 (0.0778)  loss_objectness: 0.2430 (0.2430)  loss_rpn_box_reg: 0.0240 (0.0240)  time: 10.5875  data: 2.8063  max mem: 9942
Epoch: [1]  [ 10/495]  eta: 1:03:46  lr: 0.000116  loss: 0.9003 (0.8763)  loss_classifier: 0.5021 (0.4781)  loss_box_reg: 0.0570 (0.0559)  loss_objectness: 0.2875 (0.3152)  loss_rpn_box_reg: 0.0240 (0.0271)  time: 7.8892  data: 0.2962  max mem: 9942
Epoch: [1]  [ 20/495]  eta: 1:01:24  lr: 0.000217  loss: 0.6736 (0.7047)  loss_classifier: 0.2839 (0.3309)  loss_box_reg: 0.0570 (0.0605)  loss_objectness: 0.2121 (0.2868)  loss_rpn_box_reg: 0.0164 (0.0265)  time: 7.6149  data: 0.0524  max mem: 9942
Epoch: [1]  [ 30/495]  eta: 0:59:47  lr: 0.000318  loss: 0.4141 (0.6037)  loss_classifier: 0.1338 (0.2643)  loss_box_reg: 0.0860 (0.0735)  loss_objectness: 0.1592 (0.2379)  loss_rpn_box_reg: 0.0195 (0.0279)  time: 7.6185  data: 0.0602  max mem: 9942

In [22]:
# to save model
torch.save(model.state_dict(),"/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD/road_crack/model/model")

In [23]:
from PIL import Image, ImageFont, ImageDraw
from IOU import get_iou
import matplotlib.pyplot as plt
import time

In [24]:
# Define the number of class
loaded_model = get_model(num_classes=2)
# Load the model
loaded_model.load_state_dict(torch.load("/content/RoadCrackDetection/Faster_R-CNN_Object_Detection/Pytorch_OD/road_crack/model/model"))

<All keys matched successfully>

In [26]:
def getTPR_FPR(TP, TN, FP, FN):
    TPR = 0
    FPR = 0
    if (TP == 0 and FN == 0):
        pass
    else:
        TPR = TP / (TP + FN)

    if (TN == 0 and FP == 0):
        pass
    else:
        FPR = FP / (TN + FP)

    return TPR, FPR


# calculate recall, precision, accuracy and F1 score
def getEval(TP, TN, FP, FN):
    REC = 0
    PRE = 0
    ACU = 0
    FSO = 0

    if (TP == 0 and FN == 0):
        pass
    else:
        REC = TP / (TP + FN)

    if (TP == 0 and FP == 0):
        pass
    else:
        PRE = TP / (TP + FP)

    if (TP == 0 and TN == 0 and FP == 0 and FN == 0):
        pass
    else:
        ACU = (TP + TN) / (TP + TN + FP + FN)

    if (TP == 0 and FP == 0 and FN == 0):
        pass
    else:
        FSO = (2 * TP) / ((2 * TP) + FP + FN)

    return REC, PRE, ACU, FSO


# remove the same element from the list
# to be used to check false negative
def remove_items_from_list(my_list, temp):
    dup_list = [item for item in temp if item in my_list]

    for ele in dup_list:
        my_list.remove(ele)

    return my_list


# convert class number to class name
def get_class_name(class_number_input):
    class_name = ''
    if (class_number_input == 1):
        class_name = 'Crack'
    else:
        class_name = 'NoCrack'

    return class_name

In [27]:
# visualising image and getting data for confusion matrix
def visual_image(index_of_image):
    num_total_cracks = 0
    num_cracks = 0
    num_passed_iou = 0
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    Precision = 0
    Recall = 0
    FPR = 0
    F1_score = 0
    iou_array = []
    iou_passed_class_array = []
    groundtruth_class_array = []
    extra_FP = 0
    Crack_TP = 0
    NoCrack_TP = 0

    Crack_TN = 0
    NoCrack_TN = 0

    Crack_FP = 0
    NoCrack_FP = 0

    Crack_FN = 0
    NoCrack_FN = 0

    Crack_TPR = 0
    NoCrack_TPR = 0

    Crack_FPR = 0
    NoCrack_FPR = 0

    # to calculate inference time
    start_test = time.time()
    img, _ = dataset_test[index_of_image]
    label_boxes = np.array(dataset_test[index_of_image][1]["boxes"])
    label_classes = np.array(dataset_test[index_of_image][1]["labels"])
    # print(label_classes)
    class_list = label_classes.tolist()

    # put the model in evaluation mode
    loaded_model.eval()

    with torch.no_grad():
        prediction = loaded_model([img])

    end_test = time.time()

    # print(prediction)
    pred_class_list = prediction[0]["labels"].tolist()
    # print(pred_class_list)

    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    num_groundtruth_obj = len(label_boxes)

    # draw groundtruth
    for elem in range(len(label_boxes)):
        # print(label_boxes)
        # print('Ground Truth Class', label_classes[elem])
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]), (label_boxes[elem][2], label_boxes[elem][3])],
                       outline="green", width=3)
        draw.text((label_boxes[elem][0], label_boxes[elem][1]), text=str(get_class_name(class_list[elem])))
        groundtruth_class_array.append(class_list[elem])

    for element in range(len(prediction[0]["boxes"])):
        # print(prediction[0]["boxes"])
        boxes = prediction[0]["boxes"][element].cpu().numpy()
        confidence = np.round(prediction[0]["scores"][element].cpu().numpy(), decimals=4)
        # print(prediction[0]["boxes"][element])

        # checking confidence
        if confidence >= 0.7:
            # draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline="blue", width=3)
            # draw.text((boxes[0], boxes[1]), text=(get_class_name(pred_class_list[element])))
            # print(get_class_name(pred_class_list[element]))
            prediction_class_element = pred_class_list[element]
            # print('Prediction Class', prediction_class_element)
            num_cracks += 1
            num_total_cracks += 1

            max_iou = 0
            class_name_checking = ''

            # checking IOU
            for box in range(len(label_boxes)):
                draft_cal_iou = get_iou(prediction[0]["boxes"][element], label_boxes[box])
                cal_iou = draft_cal_iou.data.cpu().numpy()
                if (cal_iou > max_iou):
                    max_iou = cal_iou
                    class_name_checking = class_list[box]
                    # print('max_iou', max_iou)

            # checking class label to meet all criteria to be a true positive
            if (max_iou >= 0.5 and prediction_class_element == class_name_checking):
                num_passed_iou += 1
                iou_array.append(max_iou)
                iou_passed_class_array.append(pred_class_list[element])
                draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline="red", width=3)
                draw.text((boxes[0], boxes[1]), text=(get_class_name(pred_class_list[element])))
                if (pred_class_list[element] == 1):
                    Crack_TP += 1
                else:
                    NoCrack_TP += 1
            else:
                # confidence is above 0.7 but it did not pass IOU and class label check
                if (pred_class_list[element] == 1):
                    Crack_FP += 1
                else:
                    NoCrack_FP += 1
        else:
            # confidence below 0.7
            TN += 1
            if (pred_class_list[element] == 1):
                Crack_TN += 1
            else:
                NoCrack_TN += 1

    # to check the possible extra false postive from true postive where the number of ground truch bounding box is lower than true positive
    if (num_passed_iou > num_groundtruth_obj):
        extra_FP = num_passed_iou - num_groundtruth_obj
        FN = 0
    else:
        FN = num_groundtruth_obj - num_passed_iou
    if (num_passed_iou > num_groundtruth_obj):
        extra_FP = num_passed_iou - num_groundtruth_obj
        TP = num_passed_iou - extra_FP
    else:
        TP = num_passed_iou

    FP = (num_cracks - num_passed_iou) + extra_FP

    if (extra_FP > 0):
        extra_FP_array = remove_items_from_list(iou_passed_class_array, groundtruth_class_array)
        # print('extra FP')
        # print(extra_FP_array)
        for element in range(len(extra_FP_array)):
            if (extra_FP_array[element] == 1):
                Crack_FP += 1
                Crack_TP -= 1
            else:
                NoCrack_FP += 1
                NoCrack_TP -= 1

    # to get class label of false negative and record it into individual class false negative
    if (FN > 0):
        FN_array = remove_items_from_list(groundtruth_class_array, iou_passed_class_array)
        # print('FN is there')
        # print(FN_array)
        for element in range(len(FN_array)):
            if (FN_array[element] == 1):
                Crack_FN += 1
            else:
                NoCrack_FN += 1

    if ((TP == 0 and FP == 0) or (TP == 0 and FN == 0) or (TN == 0 and FP == 0)):
        pass
    else:
        Precision = TP / (TP + FP)
        Recall = TP / (TP + FN)
        FPR = FP / (TN + FP)

    if (Precision == 0 and Recall == 0):
        pass
    else:
        F1_score = 2 * ((Precision * Recall) / (Precision + Recall))

    # calculate true positive rate and false positive rate for each class
    Crack_TPR, Crack_FPR = getTPR_FPR(Crack_TP, Crack_TN, Crack_FP, Crack_FN)
    NoCrack_TPR, NoCrack_FPR = getTPR_FPR(NoCrack_TP, NoCrack_TN, NoCrack_FP, NoCrack_FN)

    # display the image
    display(image)

    # output message
    if num_cracks == 1:
        print('There is', num_cracks, 'road crack in this image')
    elif num_cracks > 1:
        print('There are', num_cracks, 'road cracks in this image')
    else:
        print('There is no crack')

    infer_time = end_test - start_test

    print('TP', TP)
    print('FP', FP)
    print('FN', FN)
    print('TN', TN)
    print('Infer Time is %0.2f second' % infer_time)

    return infer_time, Recall, FPR, num_total_cracks, iou_array, TP, FP, FN, TN, Crack_TPR, NoCrack_TPR, Crack_FPR, NoCrack_FPR, Crack_TP, NoCrack_TP, Crack_TN, NoCrack_TN, Crack_FP, NoCrack_FP, Crack_FN, NoCrack_FN

In [28]:
def main():
    total = 0
    Precision = 0
    TPR_Recall = 0
    TPR_array = []
    FPR_array = []
    Crack_TPR_array = []
    Crack_FPR_array = []
    F1_score = 0
    input_Recall = 0
    input_FPR = 0
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    
    Total_Crack_TP = 0
    Total_NoCrack_TP = 0

    Total_Crack_TN = 0
    Total_NoCrack_TN = 0
    Total_Crack_FP = 0
    Total_NoCrack_FP = 0
    Total_Crack_FN = 0
    Total_NoCrack_FN = 0
    total_infer_time = 0
    all_tests = len(dataset_test)
    # some_tests = 20

    for index in range(all_tests):

        # get all recorded information and combine all values for each
        infer_time, each_Recall, each_FPR, num_total_cracks, iou_array, each_TP, each_FP, each_FN, each_TN, Crack_TPR, NoCrack_TPR, Crack_FPR, NoCrack_FPR, Crack_TP, NoCrack_TP, Crack_TN, NoCrack_TN, Crack_FP, NoCrack_FP, Crack_FN, NoCrack_FN = visual_image(
            index)
        total_infer_time += infer_time
        total += num_total_cracks
        input_Recall += each_Recall
        input_FPR += each_FPR
        TPR_array.append(each_Recall)
        FPR_array.append(each_FPR)
        Crack_TPR_array.append(Crack_TPR)
        Crack_FPR_array.append(Crack_FPR)
        TP += each_TP
        FP += each_FP
        FN += each_FN
        TN += each_TN
        Total_Crack_TP += Crack_TP
        Total_NoCrack_TP += NoCrack_TP
        Total_Crack_TN += Crack_TN
        Total_NoCrack_TN += NoCrack_TN
        Total_Crack_FP += Crack_FP
        Total_NoCrack_FP += NoCrack_FP
        Total_Crack_FN += Crack_FN
        Total_NoCrack_FN += NoCrack_FN

        # calculate recall, precision, accuracy and F1 score for each class
        Crack_Recall, Crack_Precision, Crack_Accuracy, Crack_F1_Score = getEval(Total_Crack_TP, Total_Crack_TN, Total_Crack_FP,
                                                                        Total_Crack_FN)
       # output message
        if index > 0:
            print(index + 1, 'predictions are done')
        else:
            print(index + 1, 'prediction is done')

        if len(iou_array) <= 0:
            pass
        elif len(iou_array) == 1:
            print('IOU of the bounding box of detected crack is')
        else:
            print('IOUs of the bounding boxes of detected cracks are')

        for ind in range(len(iou_array)):
            iou = float(iou_array[ind])
            print('IOU:', round(iou, 4))

        print('-----------------------------------------------------------------------------')

    if ((TP == 0 and FP == 0) or (TP == 0 and FN == 0) or (TN == 0 and FP == 0)):
        pass
    else:
        Precision = TP / (TP + FP)
        TPR_Recall = TP / (TP + FN)
        FPR = FP / (TN + FP)

    if (Precision == 0 and TPR_Recall == 0):
        pass
    else:
        F1_score = 2 * ((Precision * TPR_Recall) / (Precision + TPR_Recall))
    total_CM = TP + TN + FP + FN
    model_accuracy = (TP + TN) / total_CM

    # output message for the whole testing
    print('There are', TP, 'cracks in', len(dataset_test), 'tested images')
    print('Confusion Matrix:')
    print('True Postive is', round(TP, 4))
    print('True Negative is', round(TN, 4))
    print('False Postive is', round(FP, 4))
    print('False Negative is', round(FN, 4))
    print('Accuracy is', round(model_accuracy, 4))
    print('Precision is', round(Precision, 4))
    print('Recall is', round(TPR_Recall, 4))
    print('F1 score', round(F1_score, 4))
    print('Crack Recall is', round(Crack_Recall, 4))
    print('Crack Precision is', round(Crack_Precision, 4))
    print('Crack Accuracy is', round(Crack_Accuracy, 4))
    print('Crack F1 score is', round(Crack_F1_Score, 4))
    print('Total Inference Time is %0.2f second' % total_infer_time)
    print('Average Inference Time is %0.2f second' % (total_infer_time / all_tests))

    # sorting the all true positive and negative arrays to plot ROC curve
    TPR_array.sort()
    FPR_array.sort()
    Crack_TPR_array.sort()
    Crack_FPR_array.sort()

    # Title
    plt.title('ROC Plot')
    # Axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # plt.plot(FPR_array, TPR_array)
    plt.plot(Crack_FPR_array, Crack_TPR_array, color='blue', linewidth=3, label='Crack')
    plt.legend()
    plt.show()

In [1]:
main()

NameError: ignored